In [2]:
from config import init_env
from config import variables
import importlib
variables = importlib.reload(variables)
init_env.set_environment_variables()

# Do not modify the `assert` line below 
assert variables.RESOURCE_GROUP!='', """You should change the value assigned to the `RESOURCE_GROUP` in the `variables.py` file to your own resource group first!"""
print(f"Resource group is set to: {variables.RESOURCE_GROUP}")

Resource group is set to: default


[*Tutorial01 - Consumption of GenAI models Using Orchestration - A Beginner's Guide*](https://developers.sap.com/tutorials/ai-core-orchestration-consumption.html)
## Consumption of GenAI models Using Orchestration - A Beginner's Guide
While orchestration in SAP AI Core offers capabilities such as data masking, content filtering, translation, and grounding, this tutorial focuses on the basic consumption flow using mandatory modules like templating and model configuration. Others modules are Optional and usage of those modules are covered in a separate tutorial.
By the end of this tutorial,you will 
* Have a foundational understanding of orchestration through its minimal usage, focusing on practical application of templates and how to switch between different models using harmonized APIs.
* Learn how to implement the solution using SAP AI Launchpad, Python SDK, Java, JavaScript, and Bruno.

Refer to the [<u>orchestration documentation</u>](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/orchestration-8d022355037643cebf775cd3bf662cc5) for more information.

### Check Configuration for Orchestration deployment - Optional Step
<blockquote>
<p>In SAP AI Core, orchestration deployment is available by default in the default resource group during the onboarding. </p>
</blockquote>


To check the deployment, first need to create the client

In [5]:
import os
# Create Connection
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
ai_core_client = AICoreV2Client(
    base_url = os.environ['AICORE_BASE_URL'] + "/v2", # The present SAP AI Core API version is 2
    auth_url=  os.environ['AICORE_AUTH_URL'], 
    client_id = os.environ['AICORE_CLIENT_ID'],
    resource_group =os.environ['AICORE_RESOURCE_GROUP'],
    client_secret = os.environ['AICORE_CLIENT_SECRET']
)

Find the configuration list for orchestration 

In [6]:

response = ai_core_client.configuration.query()

# Filter items with scenario_id = 'orchestration'
orchestration_items = [vars(rg) for rg in response.resources if getattr(rg, 'scenario_id', None) == 'orchestration']

# Print details of all matching items
for item in orchestration_items:
    print("--------------- Configuration ----------------")
    for key, value in item.items():
        print(f"{key}: {value}")


--------------- Configuration ----------------
id: 796aca2d-d225-4ffe-9b0b-e83e3b3f7f9f
name: config_ss02_orchestration
scenario_id: orchestration
executable_id: orchestration
parameter_bindings: []
input_artifact_bindings: []
created_at: 2025-09-16 02:19:08+00:00
scenario: None
--------------- Configuration ----------------
id: 43b0b78c-1555-42bd-8928-4debe4f6d4fb
name: config_ss02_orchestration
scenario_id: orchestration
executable_id: orchestration
parameter_bindings: []
input_artifact_bindings: []
created_at: 2025-09-16 02:17:09+00:00
scenario: None
--------------- Configuration ----------------
id: 01d20309-52af-42b2-a368-1ea69b79aaab
name: config_ss02_orchestration
scenario_id: orchestration
executable_id: orchestration
parameter_bindings: []
input_artifact_bindings: []
created_at: 2025-09-16 02:14:49+00:00
scenario: None
--------------- Configuration ----------------
id: b4a884be-1a61-46fa-a3ed-3f8ebd5c1173
name: config_ss02_orchestration
scenario_id: orchestration
executable_id

Check the orchestration deployment using the onfiguration find from the previous step 

In [9]:
response = ai_core_client.deployment.query()

# Filter items with configuration_id  
orchestration_items = [vars(rg) for rg in response.resources if getattr(rg, 'configuration_id', None) == '796aca2d-d225-4ffe-9b0b-e83e3b3f7f9f']

# Print details of all matching items
for item in orchestration_items:
    print("--------------- Deployment ----------------")
    for key, value in item.items():
        print(f"{key}: {value}")

--------------- Deployment ----------------
id: d02c3ee65a3a9f5c
configuration_id: 796aca2d-d225-4ffe-9b0b-e83e3b3f7f9f
configuration_name: config_ss02_orchestration
scenario_id: orchestration
status: Status.RUNNING
target_status: TargetStatus.RUNNING
created_at: 2025-09-16 02:21:08+00:00
modified_at: 2025-11-14 07:14:55+00:00
status_message: None
status_details: None
submission_time: 2025-09-16 02:23:59+00:00
start_time: 2025-09-16 02:35:25+00:00
completion_time: None
deployment_url: https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d02c3ee65a3a9f5c
last_operation: Operation.CREATE
latest_running_configuration_id: 796aca2d-d225-4ffe-9b0b-e83e3b3f7f9f
details: {'scaling': {'backend_details': {}}, 'resources': {'backend_details': {}}}
ttl: None


To check the available LLM list

In [10]:
response = ai_core_client.model.query()
for rg in response.resources:
    print('model:', rg.model)
    print('allowed_scenarios', rg.allowed_scenarios)
    print(' ')

model: sonar-pro
allowed_scenarios [{'executable_id': 'perplexity-ai', 'scenario_id': 'foundation-models'}]
 
model: sonar
allowed_scenarios [{'executable_id': 'perplexity-ai', 'scenario_id': 'foundation-models'}]
 
model: cohere-reranker
allowed_scenarios [{'executable_id': 'aicore-cohere', 'scenario_id': 'foundation-models'}]
 
model: cohere--command-a-reasoning
allowed_scenarios [{'executable_id': 'aicore-cohere', 'scenario_id': 'foundation-models'}]
 
model: anthropic--claude-3-haiku
allowed_scenarios [{'executable_id': 'aws-bedrock', 'scenario_id': 'foundation-models'}, {'executable_id': 'orchestration', 'scenario_id': 'orchestration'}]
 
model: anthropic--claude-3-opus
allowed_scenarios [{'executable_id': 'aws-bedrock', 'scenario_id': 'foundation-models'}, {'executable_id': 'orchestration', 'scenario_id': 'orchestration'}]
 
model: anthropic--claude-3.5-sonnet
allowed_scenarios [{'executable_id': 'aws-bedrock', 'scenario_id': 'foundation-models'}, {'executable_id': 'orchestration

### Consume LLM's in Generative AI Hub through Orchestration

Load the CV file using the following code to read its content

In [21]:
from gen_ai_hub.orchestration.utils import load_text_file 

# Load the CV file content 
cv_file_path = "cv.txt" # Specify the correct path to the CV file 
cv_content = load_text_file(cv_file_path) 

# Print the content to verify it has been loaded 
print(cv_content) 

John Doe
1234 Data St, San Francisco, CA 94101
(123) 456-7890
johndoe@email.com
LinkedIn Profile
GitHub Profile

Objective
Detail-oriented Data Scientist with 3+ years of experience in data analysis, statistical modeling, and machine learning. Seeking to leverage expertise in predictive modeling and data visualization to help drive data-informed decision-making at [Company Name].

Education
Master of Science in Data Science
University of California, Berkeley
Graduated: May 2021

Bachelor of Science in Computer Science
University of California, Los Angeles
Graduated: May 2019

Technical Skills

Programming Languages: Python, R, SQL, Java
Data Analysis & Visualization: Pandas, NumPy, Matplotlib, Seaborn, Tableau
Machine Learning: Scikit-learn, TensorFlow, Keras, XGBoost
Big Data Technologies: Hadoop, Spark
Databases: MySQL, PostgreSQL
Version Control: Git

Professional Experience

Data Scientist
DataCorp Inc., San Francisco, CA
June 2021 – Present

Developed predictive models to optimize

The next step involves creating a template that specifies how the AI should handle the resume content. The template will include both SystemMessage and UserMessage components.

* SystemMessage: Defines the AI assistant’s role and instructions.

* UserMessage: Represents the user’s input (i.e., the CV content) to be processed by the AI. 

In [36]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage 
from gen_ai_hub.orchestration.models.template import Template, TemplateValue 

# Define the template for resume screening 
template = Template(
    messages=[
        SystemMessage(
            """You are a helpful AI assistant for HR.
            Summarize the following CV in 10 sentences,focusing on key qualifications, work experience, and achievements.
            Include personal contact information,organizational history, and personal interests"""
        ), 
        
        UserMessage(
            "Here is a candidate's resume: {{?candidate_resume}}"
        ),
    ], 

    defaults=[
        TemplateValue(
            name="candidate_resume", 
            value="John Doe's resume content goes here..."
        ), 
    ], 
)

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM 
from gen_ai_hub.orchestration.models.config import OrchestrationConfig 

# List of models to use，models have to be picked from the available list fetched before 
models = [
    LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
    LLM(name="mistralai--mistral-medium-instruct", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
    LLM(name="anthropic--claude-4.5-sonnet", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
    LLM(name="meta--llama3.1-70b-instruct", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
]

# Create configurations for each model 
configs = [] 
for model in models: 
    # Create orchestration config for each model 
    config = OrchestrationConfig( 
        template=template, 
        llm=model,
    ) 
    configs.append(config) 

print("Model configurations created successfully:") 

Model configurations created successfully:


Now, run the orchestration with the prepared configurations. The results will be saved in <i>model_responses.txt</i> for later review.

In [38]:
from gen_ai_hub.orchestration.service import OrchestrationService 

# Initialize an empty list to store the responses 
responses = [] 

# Iterate through each config and get the response using the filtered input 
for i, config in enumerate(configs):
    orchestration_service = OrchestrationService(
        #api_url=ready_deployment.deployment_url, 
        config=config
    ) 

    # Run orchestration with the provided input (for example, candidate resume content) 
    result = orchestration_service.run(
        template_values=[
            TemplateValue(
                name="candidate_resume",
                value=cv_content # Adjust 'cv_content' as needed
            )
        ]
    ) 

    # Extract the response content 
    response = result.orchestration_result.choices[0].message.content 

    # Append the response to the responses list 
    responses.append(
        { 
            "model": models[i].name, # Store model name 
            "response": response # Store the corresponding model response 
        }
    ) 

# Store the responses in a text file 
with open("model_responses.txt", "w") as file: 
    for response_data in responses: 
        file.write(f"Response from model {response_data['model']}:\n") 
        file.write(f"{response_data['response']}\n") 
        file.write("-" * 80 + "\n") # Add a separator between model responses 

[*Tutorial02 - Leveraging Orchestration Capabilities to Enhance Responses*](https://developers.sap.com/tutorials/ai-core-orchestration-consumption-opt.html)
## Leveraging Orchestration Capabilities to Enhance Responses
In this tutorial, we will explore optional orchestration capabilities available in the Gen AI Hub, such as Data Masking, translation and Content Filtering.

We will learn

☝️ Inference of GenAI models using orchestration along with Data Masking, translation and Content Filtering features

### Accessing Orchestration Capabilities
 The focus will shift from basic orchestration to leveraging optional modules to enhance data privacy and refine response quality. These enhancements include:
* Data Masking : Hiding sensitive information like phone numbers, organizational details, or personal identifiers. 

* Content Filtering : Screening for categories such as hate speech, self-harm, explicit content, and violence to ensure safe and relevant responses.

* Translation : Automatically converts input and/or output text between source and target languages to support multilingual processing.

In [40]:
from gen_ai_hub.orchestration.utils import load_text_file 
# Load the support request file content 
support_request_path = "support-request.txt" # Specify the correct path to the file 
support_request = load_text_file(support_request_path) 
# Print the content to verify it has been loaded 
print(support_request)


"Subject: Bestellung #1234567890 Verspätet - John Johnson Nachricht: Halle, ich schreibe ihnen um mich nach dem Status meiner Bestellung mit der Bestellnr. +1234567890 zu erkundigen. Die Lieferung war eigentlich für gestern geplant, ist bisher jedoch nicht erfolgt. Mein Name ist John Johnson und meine Lieferadresse lautet 125 Cole Meadows Drive Palo Alto, California 94301. Bitte lassen Sie mich per Telefon unter der Nummer +1 505802 2172 wissen, wann ich mit meiner Lieferung rechnen kann. Danke!"


### Template Configuration
The templating module is a mandatory step in orchestration. It allows you to define dynamic inputs using placeholders, construct structured prompts, and generate a final query that will be passed to the model configuration module.

In this step, we create a template that defines how the sentiment analysis prompt will be structured using message components:

* system: Defines assistant behavior and task.

* user: Provides the support request input.

Use the following code to create the template:

In [42]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

# Define the sentiment analysis template
template = Template(
    messages=[
        SystemMessage(
            """You are a customer support assistant.
             Analyze the sentiment of the user request provided and return whether the sentiment is positive, neutral, or negative. 
             Also provide a one-line justification for your classification."""
        ),
        UserMessage(
            "Please analyze the sentiment of the following support request: {{?support_text}}"
        ),
    ],
    defaults=[
        TemplateValue(
            name="support_text",
            value="User is unhappy with the latest update and facing usability issues."),
    ],
)

In [43]:
# List of models to use，models have to be picked from the available list fetched before 
models = [
    LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
    LLM(name="mistralai--mistral-medium-instruct", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
    LLM(name="anthropic--claude-4.5-sonnet", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
    LLM(name="meta--llama3.1-70b-instruct", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
]


### Setting Up Data Masking Parameters
The **Data Masking** Module ensures data privacy by anonymizing or pseudonymizing sensitive information before it is processed.

* Anonymization(匿名) : Irreversibly replaces personal identifiers with placeholders (e.g., MASKED_ENTITY). 

* Pseudonymization(假名) : Substitutes identifiers with reversible tokens (e.g., MASKED_ENTITY_ID).

For this tutorial, we use anonymization:

By following this example, you can easily integrate image-based inputs with the GPT-4o model and leverage its ability to understand and generate responses based on both visual and text content. For additional guidance, refer to the screenshot below.

<img src="https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/objectDetection.jpg" width="30%"> 

In [46]:
# Example usage
image_url = "https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/objectDetection.jpg"
text_prompt = "give me the bottle color and its count."  # Prompt asking for the description
model_name = "gpt-4o"  # Replace with the model that supports image input

# Create prompt with image and text
messages = create_image_prompt(image_url, text_prompt)

# Get response from model
response = get_response_from_model(model_name, messages)
print(response)

The bottle color is blue, and there is one bottle in the image.


### Graph Analysis
Here, the tutorial demonstrates how GPT-4o can be used to interpret and analyze data presented in graphical form. By combining text and image input, the model can extract meaningful insights from charts, graphs, and other visual data representations. This step is valuable for data analysis, reporting, and decision-making processes.

Follow the further steps to replicate graph analysis using GPT-4o.

To utilize the GPT-4o model, which supports both text and image inputs, use the code below. This example demonstrates how to create a prompt with an image URL and a text query, enabling the model to process and provide a response based on both visual and textual information.

Note: You can replace the image URL with any image of your choice and modify the text prompt to ask the model any question about that image based on your specific needs.

In [60]:
import requests
import base64

def encode_image_from_url(image_url):
    """Download and encode image to base64 format from URL."""
    response = requests.get(image_url)
    if response.status_code == 200:
        return base64.b64encode(response.content).decode("utf-8")
    else:
        raise Exception(f"Failed to download image. Status code: {response.status_code}")

In [61]:
def create_image_prompt(image_url, text_prompt):
    """Create a prompt message for the model with the image data."""
    # Encode image URL to base64 format
    image_base64 = encode_image_from_url(image_url)
    
    # Create messages including both text and image input
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": text_prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url  # Use the direct image URL
                    }
                }
            ]
        }
    ]
    return messages

In [62]:
def get_response_from_model(model_name, messages):
    """Send messages to the model and return the response."""
    kwargs = dict(model_name=model_name, messages=messages)
    response = chat.completions.create(**kwargs)
    return response.to_dict()["choices"][0]["message"]["content"]

By following this example, you can easily integrate image-based inputs with the GPT-4o model and leverage its ability to understand and generate responses based on both visual and text content. For additional guidance, refer to the screenshot below.

<img src="https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/graph.jpg" width="30%"> 

In [63]:
# Example usage
image_url = "https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/graph.jpg"
text_prompt = "what is this graph about"  # Prompt asking for the description
model_name = "gpt-4o"  # Replace with the model that supports image input

# Create prompt with image and text
messages = create_image_prompt(image_url, text_prompt)
# Get response from model
response = get_response_from_model(model_name, messages)
print(response)

The graph depicts the performance of the Dow Jones Industrial Average (DJIA) from around 2011 to 2023. The DJIA is a stock market index that represents 30 large, publicly-owned companies based in the United States. The graph shows the ups and downs of the index over this period, indicating trends, growth, and fluctuations in the market. The sharp drop around 2020 corresponds to the market impact of the COVID-19 pandemic, followed by a recovery and continued growth.


### Math
In this step, we explore how GPT-4o handles mathematical problems that involve both textual descriptions and visual data. The model can solve equations, interpret mathematical expressions in images, and provide detailed explanations of its reasoning. This capability is useful in educational tools, scientific research, and engineering applications.

Follow the further steps to replicate mathematical operations using GPT-4o.

To utilize the GPT-4o model, which supports both text and image inputs, use the code below. This example demonstrates how to create a prompt with an image URL and a text query, enabling the model to process and provide a response based on both visual and textual information.

Note: You can replace the image URL with any image of your choice and modify the text prompt to ask the model any question about that image based on your specific needs.

In [52]:
import requests
import base64

def encode_image_from_url(image_url):
    """Download and encode image to base64 format from URL."""
    response = requests.get(image_url)
    if response.status_code == 200:
        return base64.b64encode(response.content).decode("utf-8")
    else:
        raise Exception(f"Failed to download image. Status code: {response.status_code}")

In [53]:
def create_image_prompt(image_url, text_prompt):
    """Create a prompt message for the model with the image data."""
    # Encode image URL to base64 format
    image_base64 = encode_image_from_url(image_url)
    
    # Create messages including both text and image input
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": text_prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url  # Use the direct image URL
                    }
                }
            ]
        }
    ]
    return messages

In [54]:
def get_response_from_model(model_name, messages):
    """Send messages to the model and return the response."""
    kwargs = dict(model_name=model_name, messages=messages)
    response = chat.completions.create(**kwargs)
    return response.to_dict()["choices"][0]["message"]["content"]

By following this example, you can easily integrate image-based inputs with the GPT-4o model and leverage its ability to understand and generate responses based on both visual and text content. For additional guidance, refer to the screenshot below.

<img src="https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/math.jpg" width="20%"> 

In [55]:
# Example usage
image_url = "https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/math.jpg"
text_prompt = "find x"  # Prompt asking for the description
model_name = "gpt-4o"  # Replace with the model that supports image input

# Create prompt with image and text
messages = create_image_prompt(image_url, text_prompt)

# Get response from model
response = get_response_from_model(model_name, messages)
print(response)

To solve the equation \((2x - 10)/2 = 3(x - 1)\), follow these steps:

1. Multiply both sides by 2 to eliminate the fraction:
   \[
   2x - 10 = 6(x - 1)
   \]

2. Distribute the 6 on the right-hand side:
   \[
   2x - 10 = 6x - 6
   \]

3. Rearrange the equation to bring all terms involving \(x\) on one side and constant terms on the other side. Subtract \(2x\) from both sides:
   \[
   -10 = 4x - 6
   \]

4. Add 6 to both sides to isolate the term with \(x\):
   \[
   -4 = 4x
   \]

5. Divide both sides by 4 to solve for \(x\):
   \[
   x = -1
   \]

Thus, the solution is \(x = -1\).


### Image to Text
The final step focuses on converting visual information into text. By providing an image as input, GPT-4o generates a textual description or transcription of the content. This step is particularly beneficial for accessibility tools, content creation, and archiving visual data.

Follow the further steps to replicate Optical Character Recognition (OCR) using GPT-4o.

To utilize the GPT-4o model, which supports both text and image inputs, use the code below. This example demonstrates how to create a prompt with an image URL and a text query, enabling the model to process and provide a response based on both visual and textual information.

Note: You can replace the image URL with any image of your choice and modify the text prompt to ask the model any question about that image based on your specific needs.

In [ ]:
import requests
import base64
def encode_image_from_url(image_url):
    """Download and encode image to base64 format from URL."""
    response = requests.get(image_url)
    if response.status_code == 200:
        return base64.b64encode(response.content).decode("utf-8")
    else:
        raise Exception(f"Failed to download image. Status code: {response.status_code}")

In [57]:
def create_image_prompt(image_url, text_prompt):
    """Create a prompt message for the model with the image data."""
    # Encode image URL to base64 format
    image_base64 = encode_image_from_url(image_url)
    
    # Create messages including both text and image input
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": text_prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url  # Use the direct image URL
                    }
                }
            ]
        }
    ]
    return messages

In [58]:
def get_response_from_model(model_name, messages):
    """Send messages to the model and return the response."""
    kwargs = dict(model_name=model_name, messages=messages)
    response = chat.completions.create(**kwargs)
    return response.to_dict()["choices"][0]["message"]["content"]

By following this example, you can easily integrate image-based inputs with the GPT-4o model and leverage its ability to understand and generate responses based on both visual and text content. For additional guidance, refer to the screenshot below.

<img src="https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/handwrittenText.png" width="45%"> 

In [59]:
# Example usage
image_url = "https://raw.githubusercontent.com/SAP-samples/ai-core-samples/main/09_BusinessAIWeek/images/handwrittenText.png"
text_prompt = "extract text"  # Prompt asking for the description
model_name = "gpt-4o"  # Replace with the model that supports image input

# Create prompt with image and text
messages = create_image_prompt(image_url, text_prompt)

# Get response from model
response = get_response_from_model(model_name, messages)
print(response)

Dear User,

Handwrytten uses robotic handwriting
machines that use an actual pen to
write your message. The results are
virtually indistinguishable from actual
handwriting.
Try it today!

The Robot
